<a href="https://colab.research.google.com/github/aiatoz/NLP/blob/main/WebQueryEngine/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is an open source project to ground an LLM on Web Data for querying

Due to limited compute I've restricted the context length to 2048. Feel free to increase it if you've enough computing power. The default context length of 2048 will work just fine with a T4 GPU

For more details reagrding the libraries used, please refer

Url lib : https://pypi.org/project/urllib3/

BeautifulSoup : https://pypi.org/project/beautifulsoup4/

ctransformers : https://pypi.org/project/ctransformers/

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 28 14:18:34 2023

@author: Krishna
"""


#Installing dependencies

!pip install urllib3
!pip install ctransformers[cuda]

Scraping and filtering out data

In [14]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib3


def filterTag(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def htmlToText(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.find_all(string=True)
    visible_texts = filter(filterTag, texts)
    return u" ".join(t.strip() for t in visible_texts)

req = urllib3.PoolManager()
response = req.request('GET', "https://ai.google/")
htmlContent = response.data

scrapedText = htmlToText(htmlContent)
#print(scrapedText)
print(scrapedText)

                        AI               AI    Why AI       Responsibility             Responsibility      AI Principles  Objectives for building beneficial AI        Responsible AI Practices  Guiding responsible AI development        AI Governance & Operations  Our review and approval process        Social Good  Addressing societal challenges with AI        Policy  Our contributions to AI Governance          FEATURED RESEARCH   AI for the benefit of humanity      EXAMPLES OF OUR WORK   Improving skin tone evaluation in machine learning to uphold our AI principles           Discover             Discover    Ai across Google   Foundation Models  Learn more about our models    Generative AI  How teams at Google are using AI    PaLM 2  Next generation large language model    Quantum AI  Building a useful quantum computer        Learn   Research  AI publications, tools, and datasets        Blogs   AI Blogs  Current news and stories from Google            FEATURED CONTENT   Learn more about 

#Loading and using the LLM

The LLM used here is Mistral 7b which is a light LLM that can even run on a consumer end GPU

In [15]:
from ctransformers import AutoModelForCausalLM
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type = "mistral", gpu_layers = 50, context_length=2048)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
query = input("Enter the question :")

Content = """<s>[INST]
The data enclosed in tilde symbol is a web scraped output. Avoid the unnecessary parts such as 'Jump to content', 'Main menu', 'Navigation' etc,
that usually occures in websites.

Based on the data answer the queries. The answers needs to be short and simple, and don't have to mention the question again in the answer

Web Data : ~
{}
~

Query : {}

[/INST]""".format(scrapedText, query)

#To enable chat, you can just append the messages, the context will act like a memory, and clearing it will automatically wipe the memory out
#print(llm(Content))

print(llm(Content))

Enter the question :Write a summary
 Google is using AI to develop and harness artificial intelligence for the benefit of humanity. They are committed to responsible innovation and technologies that benefit all of humanity. Their mission is to organize the world's information and make it universally accessible and useful. They are working to develop AI responsibly to benefit people and society by exploring ways in which they can apply AI to create innovative new products and services, improve their user experience and provide valuable social impact through their work.
